### Check where environment is set up

In [1]:
#use miniconda-analysis environment
import sys

sys.executable

#Below installs rasterio package so that I can read in GeoTIFF files. One-time only
#conda install -c conda-forge rasterio

'/glade/u/home/dll/miniconda/envs/analysis/bin/python'

### Import packages

In [2]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import xarray as xr
import xesmf as xe
import matplotlib.pyplot as plt
import metpy as metpy
import pandas as pd
import pathlib
import netCDF4
from netCDF4 import Dataset
from ctsm_py import utils
from ctsm_py import regrid_utils

### Importing SIF data

In [3]:
root_path = pathlib.Path('/glade/work/dll/SIF_GPP/data.globalecology.unh.edu/data/GOSIF-GPP_v2/Monthly/Mean/')
files = sorted(root_path.rglob("*.tif"))

In [4]:
dsets = []
for file in files:
    da = xr.open_rasterio(file, chunks={})
    dsets.append(da)

ds = xr.concat(dsets, dim='sif_gpp')

#2000 isn't a full year, removing 
ds = ds[10:]

In [5]:
ds

<xarray.DataArray (sif_gpp: 216, band: 1, y: 3600, x: 7200)>
dask.array<shape=(216, 1, 3600, 7200), dtype=uint16, chunksize=(1, 1, 3600, 7200)>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 89.97 89.92 89.88 89.83 ... -89.83 -89.88 -89.93 -89.98
  * x        (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.8 179.9 179.9 180.0
Dimensions without coordinates: sif_gpp
Attributes:
    transform:   (0.05, 0.0, -180.0, 0.0, -0.05, 90.0)
    crs:         +init=epsg:4326
    res:         (0.05, 0.05)
    is_tiled:    1
    nodatavals:  (nan,)

### CLM data

In [6]:
simyrs             = "185001-201412"
var                = "GPP"

datadir            = "/glade/p/cesm/lmwg_dev/dll/"
subdir             = "/lnd/proc/tseries/month_1/"
Mod1dir            = "CESM2_Coupled_NoCrop/"
Mod2dir            = "CESM2_Coupled_NoFert/"

sim                = "b.e21.BHIST_BPRP.f09_g17.CMIP6-esm-hist.001"
sim2               = "b40.20th.1deg.coup.001"
sim3               = "b.e21.BHIST_BPRP.f09_g17.CMIP6-esm-hist.002_NoCropSpin_Transient1970-2014"
sim4               = "b.e21.BHIST_BPRP.f09_g17.CMIP6-esm-hist.002_NoFert_Transient1970-2014"

In [7]:
#need to grab GPP data for the CMIP simulations before I can use them
#data1          = utils.time_set_mid(xr.open_dataset(datadir+Mod1dir+sim+".clm.h0."+var+"."+simyrs+".nc", decode_times=True), 'time')
#data2          = utils.time_set_mid(xr.open_dataset(datadir+Mod1dir+sim2+".clm2.h0."+var+".185001-200512.nc", decode_times=True), 'time')
#data3          = utils.time_set_mid(xr.open_dataset(datadir+Mod1dir+subdir+sim3+".clm2.h0."+var+".197001-201412.nc", decode_times=True), 'time')
data4          = utils.time_set_mid(xr.open_dataset(datadir+Mod2dir+subdir+sim4+".clm2.h0."+var+".197001-201412.nc", decode_times=True), 'time')

In [8]:
data4 = data4.assign({'lon': (['lon'],xr.where(data4.lon<0, 360+data4.lon, data4.lon))})
data4 = data4.sortby('lon', ascending=True)
data4 = data4.sortby('lat', ascending=True)

## Attempting regridding using ESMF tools
#### Note that I don't have this working. Attempted method from M. Long below (also not working).

In [9]:
#creating new grid for sif data
sif_coarse = xr.DataArray({'lat': (['lat'], data4.lat),
                     'lon': (['lon'], data4.lon)})
sif_coarse

<xarray.DataArray ()>
array(<built-in method values of dict object at 0x2b57567a82d0>, dtype=object)

In [10]:
#Rename sif coordinates to lat and lon. Can only do this once 
sifdat_0 = ds.rename({'x':'lon'})
sifdat = sifdat_0.rename({'y':'lat'})

#'assign' doesn't work here, so can't rework the longitude coordinate to 0-360.
#sifdat = sifdat.assign({'lon': (['lon'],xr.where(sifdat.lon<0, 360+sifdat.lon, sifdat.lon))})
#Tested whether this makes a difference in regridder error below -- it doesn't
sifdat = sifdat.sortby('lon', ascending=True)
sifdat = sifdat.sortby('lat', ascending=True)

In [11]:
#Note: Is the error below because 'lat' and 'lon' are float not integers? 
#These don't seem to actually convert coordinate to integer values
#Not sure it makes sense to convert to integers, either. 
#sifdat_ind = sifdat.lat.astype(int)
#sifdat.lon.astype(int)
#sifdat_ind

In [12]:
sifdat = sifdat.squeeze()
#sifdat

In [13]:
#Tested this code with first regridder option below; same error
sifdat = np.asarray(sifdat)
#sifdat

In [14]:
#Tested whether using only a slice makes a different in the regridder -- it doesn't
sifdat_gen_weights = sifdat[0]
sifdat_gen_weights

array([[65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       ...,
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535]], dtype=uint16)

In [15]:
%%time
#sif_coarse = xr.Dataset({'lat': (['lat'], np.arange(-90,90,0.94)),
#                         'lon': (['lon'], np.arange(0,360,1.25)),
#                        }
#                       )
#sif_coarse

#Not sure why options below won't work. Even IndexError seems to point to 'lon' variable
#Somehow, something is wrong with the lat/lon in the input or output (assume input)

#Below results in error: KeyError: 'lon'
#regridder = xe.Regridder(sifdat, sif_coarse, 'bilinear')

#Below results in: IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
#regridder = xe.Regridder(np.asarray(sifdat), sif_coarse, 'bilinear')

#Below results in: IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
regridder = xe.Regridder(sifdat_gen_weights, sif_coarse, 'bilinear')

regridder

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [16]:
sif_out = regridder(sif_coarse)
sif_out

NameError: name 'regridder' is not defined

## Regridding using method from Matt Long
#### Note that I don't have this working properly.

In [17]:
#From Matt Long
# https://github.com/marbl-ecosys/forcing-Fe-sedflux/blob/mom6-dev/notebooks/regrid_tools/core.py
class regridder(object):
    """simple class to enable regridding"""
    
    def __init__(self, src_grid, dst_grid, method='bilinear', clobber=False):
        
        self.src_grid_file = src_grid.grid_file
        self.dst_grid_file = dst_grid.grid_file
        self.method = method
        self.weight_file = f'{regrid_dir}/{src_grid.grid_name}_to_{dst_grid.grid_name}_{method}.nc'   
        # below line attempting to hard code directory
        #        self.weight_file = f'/glade/work/dll/esmlab-regrid/weights/sifgrid_latlon_0.05x0.05_180W_to_CESM_latlon_1x1_180W_conservative.nc'
        esmf_regrid_weight_gen(
            self.src_grid_file, self.dst_grid_file, self.weight_file, self.method, clobber
        )
                
        self.dims_src = src_grid.dims
        self.dims_dst = dst_grid.dims
        self.mask_dst = dst_grid.mask

        n_dst = np.prod(self.dims_dst)
        n_src = np.prod(self.dims_src)
        print(f'source grid dims: {self.dims_src}')
        print(f'destination grid dims: {self.dims_dst}')

        with xr.open_dataset(self.weight_file) as mf:
            row = mf.row.values - 1
            col = mf.col.values - 1
            S = mf.S.values
        self.weights = sps.coo_matrix((S, (row, col)), shape=[n_dst, n_src])

    def __repr__(self):
        return (
            f'regridder {os.path.basename(self.src_grid_file)} --> {os.path.basename(self.dst_grid_file)}'
        )
    
    def _regrid_dataarray(self, da_in, renormalize=True, apply_mask=True):
        """regrid DataArray"""
        # Pull data, dims and coords from incoming DataArray
        data_src = da_in.data
        non_lateral_dims = da_in.dims[:-2]
        copy_coords = {d: da_in.coords[d] for d in non_lateral_dims if d in da_in.coords}

        # If renormalize == True, remap a field of ones
        if renormalize:
            ones_src = np.where(np.isnan(data_src), 0.0, 1.0)
            data_src = np.where(np.isnan(data_src), 0.0, data_src)

        # remap the field
        data_dst = esmf_apply_weights(
            self.weights, data_src, self.dims_src, self.dims_dst
        )

        # Renormalize to include non-missing data_src
        # TODO: it would be nice to include a threshold here,
        #       the user could specify a fraction of mapped points, 
        #       below which the value yields missing in the data_dst
        if renormalize:
            old_err_settings = np.seterr(invalid='ignore')
            ones_dst = esmf_apply_weights(
                self.weights, ones_src, self.dims_src, self.dims_dst
            )
            ones_dst = np.where(ones_dst > 0.0, ones_dst, np.nan)
            data_dst = data_dst / ones_dst
            data_dst = np.where(ones_dst > 0.0, data_dst, np.nan)
            np.seterr(**old_err_settings)

        # reform into xarray.DataArray
        da_out = xr.DataArray(
            data_dst, name=da_in.name, dims=da_in.dims, attrs=da_in.attrs, coords=copy_coords
        )

        # Apply a missing-values mask
        if apply_mask:
            da_out = da_out.where(self.mask_dst.T)

        return da_out
    
    def regrid(self, obj, **kwargs):
        """generic interface for regridding DataArray or Dataset"""
        if isinstance(obj, xr.Dataset):
            return obj.map(self._regrid_dataarray, keep_attrs=True, **kwargs)
        elif isinstance(obj, xr.DataArray):
            return self._regrid_dataarray(obj, **kwargs)
        else:
            raise ValueError('unknown type')

In [18]:
#Pointing to files I created using pbs script
# /glade/work/dll/CTSM_py/notebooks/esmf_gen_weights_sif_to_CESM.pbs

regrid_dir = f'/glade/work/dll/esmlab-regrid/weights'
src_grid   = 'sifgrid_latlon_0.05x0.05_180W'
dst_grid   = 'CESM_latlon_1x1_180W'
method     = 'conserve'

In [19]:
grid_file  = 'sifgrid_latlon_0.05x0.05_180W_to_CESM_latlon_1x1_180W_conservative.nc'
regridder  = regrid_utils.regridder(src_grid, dst_grid, method, clobber=False)

AttributeError: 'str' object has no attribute 'grid_file'

In [20]:
# Use this as a template for regridding:
# https://github.com/marbl-ecosys/forcing-Fe-sedflux/blob/mom6-dev/notebooks/preliminary_MOM6_forcing.ipynb

sif_1deg = regridder.regrid(ds, renormalize=True, apply_mask=True)
sif_1deg

TypeError: regrid() missing 1 required positional argument: 'obj'

### Reshape data to year and month (necessary to calculate the max, amplitude, &/or annual cycle)

In [38]:
#sifdata = xr.DataArray(ds, coords=[sif_gpp, y, x], dims=["time", "lat", "lon"])
#ds.sif_gpp
#print(ds.y)

sif = pd.DataFrame(ds,columns = ['GPP','band','lat','lon'])
sifyr = sif.reshape(-1,12)

#This works, but loses metadata (lat, lon, etc.)
#sif = (ds.sif_gpp.size//12, 12)
#setattr(sif)


sif
sifyr


#OTHER ATTEMPTS -- NOT WORKING

#ds.sif_gpp.size
#sif = ds.set_index(['x', 'y'])[['sif_gpp']].to_xarray()

#sif = ds.resample(sif_gpp=12)
#sif = xr.DataArray(sif, coords=[("lat", ds.y)])
#sif


#This doesn't work -- 'DataArray' object has no attribute 'reshape'
#sif = ds.reshape(18,12)

#sifdata = xr.DataArray(ds, coords=[("lat", y)])
#sifdata


#sifmax  = ds.resample(sif_gpp=12).max()

KeyboardInterrupt: 

#### Importing single SIF data file

In [ ]:
#0.05x0.05 resolution
sifdir = "/glade/work/dll/SIF_GPP/data.globalecology.unh.edu/data/GOSIF-GPP_v2/Monthly/Mean/"
sifdat = xr.open_rasterio(sifdir + "GOSIF_GPP_2001.M01_Mean.tif")
sifdat

In [ ]:
#Rename coordinates to lat and lon. Can only do this once 
sifdat = sifdat.rename({'x':'lon'})
sifdat = sifdat.rename({'y':'lat'})

#'assign' doesn't work here, so can't rework the longitude coordinate to 0-360.
#sifdat = sifdat.assign({'lon': (['lon'],xr.where(sifdat.lon<0, 360+sifdat.lon, sifdat.lon))})
sifdat = sifdat.sortby('lon', ascending=True)
sifdat = sifdat.sortby('lat', ascending=True)

sifdat
#units: g C m^-2 mo^-1

In [ ]:
sifdat.squeeze().plot.imshow()